In [1]:
import pandas as pd
from pathlib import Path
import re

# --- CONFIG
INPUT_HT = "/media/sagarkumar/New Volume/SAGAR/2-year-data/HTCABLE_Clean.csv"
INPUT_ENERGY = "/media/sagarkumar/New Volume/SAGAR/2-year-data/ENERGYAUDIT.csv"
OUTPUT_PATH = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_trace_22_33kv.csv"

FEEDER_ID_COL = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL = "SOURCE_SSFL"
DST_LOC_COL = "DESTINATION_SSFL"
FUNC_LOC_COL = "FUNC_LOC"
DATE_COL = "SYSTEM_DATE"
LOAD_COL = "MD_KVA"

# --- LOAD HTCABLE.csv
ht = pd.read_csv(INPUT_HT, low_memory=False)

# Function to extract feeder token for 22kV/33kV only (case-insensitive)
def feeder_token_22_33(val):
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.upper().split("_")
    if len(p) >= 3 and p[1] in {'22KV', '33KV'}:
        return p[2]
    return None

ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(feeder_token_22_33)
ht = ht[ht["FEEDER_ID"].notna()]  # Keep only 22kV/33kV feeders

# Build FROM_TO column as "SRC_SWITCH_ID-DST_SWITCH_ID"
ht["FROM_TO"] = ht[SRC_SWITCH_COL].astype(str) + "-" + ht[DST_SWITCH_COL].astype(str)

# --- LOAD ENERGYAUDIT.csv and aggregate
audit = pd.read_csv(INPUT_ENERGY, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]
audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])
agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# --- MERGE by DESTINATION_SSFL = FUNC_LOC
ht[SRC_LOC_COL] = ht[SRC_LOC_COL].astype(str)
ht[DST_LOC_COL] = ht[DST_LOC_COL].astype(str)
merged = ht.merge(agg, how="left", left_on=DST_LOC_COL, right_on=FUNC_LOC_COL)

# --- OUTPUT ONLY NEEDED COLUMNS
final_cols = [
    "FEEDER_ID",
    "FROM_TO",
    SRC_LOC_COL,
    DST_LOC_COL,
    "LATEST_DT_DATE",
    "DT_LOAD"
]
merged = merged[final_cols]

# --- REMOVE DUPLICATE ROWS
merged = merged.drop_duplicates()

# --- EXPORT
merged.to_csv(OUTPUT_PATH, index=False)
print(f"Saved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass


Saved 933 rows → /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_trace_22_33kv.csv


,FEEDER_ID,FROM_TO,SOURCE_SSFL,DESTINATION_SSFL,LATEST_DT_DATE,DT_LOAD
0,220AAR02,220AAR92-33099,1S-MH-MU-ZSC-CL10-3581,1S-MH-MU-ZET-RSTN-HIRA,2025-02-07,200.446784
2,220AAR05,220AAR05-CAP BANK,1T-MH-MU-ARY,1T-MH-MU-ARY,NaT,NaN
3,220AAR100,220AAR100-CAP BANK,1T-MH-MU-ARY,1T-MH-MU-ARY,NaT,NaN
4,220AAR102,220AAR102-33008,1T-MH-MU-ARY,1S-MH-MU-ZST-RSTN-VILE,2011-08-17,36.819512
75,220AAR102,11435-31090,1S-MH-MU-ZST-RSTN-VILE,1S-MH-MU-ZST-RSTN-VILE,2011-08-17,36.819512


NOT 11 33 and 22 KV

In [2]:
import pandas as pd

# --- CONFIG
INPUT_HT = "/media/sagarkumar/New Volume/SAGAR/2-year-data/HTCABLE_Clean.csv"
INPUT_ENERGY = "/media/sagarkumar/New Volume/SAGAR/2-year-data/ENERGYAUDIT.csv"
OUTPUT_PATH = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_trace_not_11_22_33kv_fullid.csv"

FEEDER_ID_COL = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL = "SOURCE_SSFL"
DST_LOC_COL = "DESTINATION_SSFL"
FUNC_LOC_COL = "FUNC_LOC"
DATE_COL = "SYSTEM_DATE"
LOAD_COL = "MD_KVA"

# --- LOAD HTCABLE.csv
ht = pd.read_csv(INPUT_HT, low_memory=False)

# Only filter: keep full FEEDERID if NOT 11KV/22KV/33KV anywhere in string
def is_not_11_22_33kv(feederid):
    if not isinstance(feederid, str):
        feederid = str(feederid) if feederid is not None else ""
    s = feederid.upper()
    if any(k in s for k in ['_11KV_', '_22KV_', '_33KV_']):
        return False
    return True

ht = ht[ht[FEEDER_ID_COL].apply(is_not_11_22_33kv)].copy()
ht["FEEDER_ID"] = ht[FEEDER_ID_COL]  # Just copy the full original value

# Build FROM_TO column as "SRC_SWITCH_ID-DST_SWITCH_ID"
ht["FROM_TO"] = ht[SRC_SWITCH_COL].astype(str) + "-" + ht[DST_SWITCH_COL].astype(str)

# --- LOAD ENERGYAUDIT.csv and aggregate
audit = pd.read_csv(INPUT_ENERGY, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]
audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])
agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# --- MERGE by DESTINATION_SSFL = FUNC_LOC
ht[SRC_LOC_COL] = ht[SRC_LOC_COL].astype(str)
ht[DST_LOC_COL] = ht[DST_LOC_COL].astype(str)
merged = ht.merge(agg, how="left", left_on=DST_LOC_COL, right_on=FUNC_LOC_COL)

# --- OUTPUT ONLY NEEDED COLUMNS
final_cols = [
    "FEEDER_ID",
    "FROM_TO",
    SRC_LOC_COL,
    DST_LOC_COL,
    "LATEST_DT_DATE",
    "DT_LOAD"
]
merged = merged[final_cols]

# --- REMOVE DUPLICATE ROWS
merged = merged.drop_duplicates()

# --- EXPORT
merged.to_csv(OUTPUT_PATH, index=False)
print(f"Saved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass


Saved 969 rows → /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_trace_not_11_22_33kv_fullid.csv


,FEEDER_ID,FROM_TO,SOURCE_SSFL,DESTINATION_SSFL,LATEST_DT_DATE,DT_LOAD
0,NaN,29916-16915,1S-MH-MU-ZSC-CL08-3108,1S-MH-MU-ZSC-CL08-2843,2025-04-04,388.498522
2,NaN,11292-2052,nan,1S-MH-MU-ZET-CL06-8690,2025-04-04,192.449655
3,NaN,3136-14325,1S-MH-MU-ZET-RSTN-SAKI,1S-MH-MU-ZSC-RSTN-AARE,2025-04-04,238.494349
4,NaN,3136-14325,1S-MH-MU-ZET-RSTN-SAKI,1T-MH-MU-ARY,NaT,NaN
29,NaN,25270-25271,1S-MH-MU-ZET-RSTN-CHEM,1S-MH-MU-ZET-RSTN-CHEM,2025-02-07,462.579397
